In [1]:
import os
import pandas as pd
import numpy as np

In [14]:
# 아파트 관련 데이터 불러오기
df1 = pd.read_csv("2019년_서울_아파트매매_실거래가.csv", encoding = "cp949")
df2 = pd.read_csv("2019년_서울시_아파트주소.csv", encoding = "cp949")

In [15]:
df1.head()

,거래일자,법정동,도로명,아파트,층,전용면적,건축년도,거래금액
0,2019-10-05,면목동,용마산로,현대,2,84.81,1994,52000
1,2019-10-07,면목동,중랑천로,신성,5,59.91,1998,37000
2,2019-10-09,면목동,용마산로,현대,19,84.09,1994,57500
3,2019-10-10,면목동,사가정로41길,동원베네스트,5,84.86,2003,56000
4,2019-10-14,면목동,용마산로,현대,9,72.63,1994,50500


In [16]:
df2.head()

,시군구명,읍면동명,도로명,건물명,경도,위도
0,종로구,신교동,필운대로,월드빌,126.966744,37.586644
1,종로구,숭인동,숭인동1길,삼성타운,127.017780,37.579576
2,종로구,숭인동,숭인동1나길,삼성타운,127.018026,37.579582
3,종로구,평창동,평창6길,월드빌,126.973484,37.620055
4,중구,장충동1가,장충단로6가길,삼우빌라,127.006113,37.563739


In [18]:
# 데이터 병합
df = pd.merge(df1, df2,left_on = ['법정동', '도로명', '아파트'], right_on = ['읍면동명', '도로명', '건물명'])
df

,거래일자,법정동,도로명,아파트,층,전용면적,건축년도,거래금액,시군구명,읍면동명,건물명,경도,위도
0,2019-10-23,면목동,겸재로,동서그랜드맨션,4,131.34,1993,43000,중랑구,면목동,동서그랜드맨션,127.075856,37.588817
1,2019-10-25,면목동,면목로,킴스아파트,4,38.91,2018,28900,중랑구,면목동,킴스아파트,127.084734,37.592972
2,2019-10-29,면목동,용마산로86길,면목동2차미소지움아파트,12,78.56,2005,45400,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641
3,2019-12-04,면목동,용마산로86길,면목동2차미소지움아파트,4,81.96,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641
4,2019-12-26,면목동,용마산로86길,면목동2차미소지움아파트,3,82.28,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,2019-11-11,망원동,방울내로11길,동보프리미아,9,63.02,2004,39000,마포구,망원동,동보프리미아,126.894313,37.562626
1479,2019-12-04,신공덕동,백범로,신공덕e편한세상,2,59.62,2007,89900,마포구,신공덕동,신공덕e편한세상,126.954378,37.544351
1480,2019-12-07,신공덕동,백범로,신공덕e편한세상,4,59.58,2007,92500,마포구,신공덕동,신공덕e편한세상,126.954378,37.544351
1481,2019-12-16,신공덕동,백범로,신공덕e편한세상,11,59.85,2007,100000,마포구,신공덕동,신공덕e편한세상,126.954378,37.544351


In [19]:
df3 = pd.read_excel("지하철역_위경도.xlsx")

<ipython-input-19-f910b1731380>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df3 = pd.read_excel("지하철역_위경도.xlsx")


In [20]:
df3.head()

,역명,위도,경도
0,가양역[9호선],37.561560,126.854261
1,완정역[2호선],37.592867,126.673027
2,증미역[9호선],37.558144,126.860617
3,마전역[2호선],37.597649,126.667201
4,등촌역[9호선],37.550694,126.865540


In [21]:
# 거리 행렬 생성을 위한 컬럼 추출
df_location = df[['경도', '위도']]
df3_location = df3[['경도', '위도']]

In [22]:
# 거리 행렬 생성
from scipy.spatial.distance import cdist
from haversine import haversine

# 두 테이블간의 거리계산, 가까운역까지의 거리
dist_mat = cdist(df_location, df3_location, metric = haversine)
# 'euclidean', 'cityblock', 'jaccard'

In [23]:
#df_location의 각 행별로 df3와 가까운역 인덱스가 저장되어있다.
close_subway_index = dist_mat.argsort()[:, 0] # 0번째 컬럼까지 가져옴
df['가까운역'] = df3.iloc[close_subway_index]['역명'].values 
# 새로운 시리즈를 만들 때는 list, ndarray를 사용하는 것이 바람직하다!
df['가까운역까지_거리'] = dist_mat[close_subway_index][:, 0]

In [24]:
df.head()

,거래일자,법정동,도로명,아파트,층,전용면적,건축년도,거래금액,시군구명,읍면동명,건물명,경도,위도,가까운역,가까운역까지_거리
0,2019-10-23,면목동,겸재로,동서그랜드맨션,4,131.34,1993,43000,중랑구,면목동,동서그랜드맨션,127.075856,37.588817,중랑역[경춘선],22.308503
1,2019-10-25,면목동,면목로,킴스아파트,4,38.91,2018,28900,중랑구,면목동,킴스아파트,127.084734,37.592972,상봉역[7호선],25.366110
2,2019-10-29,면목동,용마산로86길,면목동2차미소지움아파트,12,78.56,2005,45400,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641,망우역[경의중앙선],25.379852
3,2019-12-04,면목동,용마산로86길,면목동2차미소지움아파트,4,81.96,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641,망우역[경의중앙선],25.379852
4,2019-12-26,면목동,용마산로86길,면목동2차미소지움아파트,3,82.28,2005,45500,중랑구,면목동,면목동2차미소지움아파트,127.094170,37.589641,망우역[경의중앙선],25.379852
